In [1]:
import os
import rasterio
import numpy as np
import tkinter as tk
from PIL import Image, ImageTk

In [ ]:
input_folder = "../image_tiles/ground_truth_cd_tif"
output_folder = "../image_tiles/ground_truth_cd_png"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".tif"):
        input_path = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + ".png"
        output_path = os.path.join(output_folder, output_filename)

        with rasterio.open(input_path) as src:
            data = src.read(1)

        grayscale_array = (data * 255).astype(np.uint8)
        image = Image.fromarray(grayscale_array, mode='L')
        image.save(output_path)

In [ ]:
input_folder = "../image_tiles/optical_tif"
output_folder = "../image_tiles/optical_png"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".tif"):
        input_path = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + ".png"
        output_path = os.path.join(output_folder, output_filename)

        with rasterio.open(input_path) as src:
            data = src.read()
            rgb_array = np.transpose(data[:3], (1, 2, 0)).astype(np.uint8)

        image = Image.fromarray(rgb_array, mode='RGB')
        image.save(output_path)

In [ ]:
input_folder = "../image_tiles/sar_tif"
output_folder = "../image_tiles/sar_png"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(".tif"):
        input_path = os.path.join(input_folder, filename)
        output_filename = os.path.splitext(filename)[0] + ".png"
        output_path = os.path.join(output_folder, output_filename)

        with rasterio.open(input_path) as src:
            data = src.read()
            rgb_array = np.transpose(data[:3], (1, 2, 0)).astype(np.uint8)

        image = Image.fromarray(rgb_array, mode='RGB')
        image.save(output_path)

In [2]:
folders = ["../image_tiles/ground_truth_cd_png", "../image_tiles/optical_png", "../image_tiles/sar_png"]
for folder in folders:
    folder_path = os.path.join(os.getcwd(), folder)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(".png"):
            file_path = os.path.join(folder_path, filename)
            image = Image.open(file_path)
            if np.array(image).sum() == 0:
                os.remove(file_path)
                print(f"Removed {file_path}")

In [ ]:
sar_folder = "../image_tiles/sar_png"
optical_folder = "../image_tiles/optical_png"
ground_truth_folder = "../image_tiles/ground_truth_cd_png"

output_folders = {
    "optical": "../data_cd/optical",
    "sar": "../data_cd/sar",
    "ground_truth": "../data_cd/ground_truth"
}

for folder in output_folders.values():
    os.makedirs(folder, exist_ok=True)

sar_files = set(os.listdir(sar_folder))
optical_files = set(os.listdir(optical_folder))
ground_truth_files = set(os.listdir(ground_truth_folder))

common_files = sar_files & optical_files & ground_truth_files

for filename in common_files:
    img = Image.open(os.path.join(optical_folder, filename))
    img.save(os.path.join(output_folders["optical"], filename))

    img = Image.open(os.path.join(ground_truth_folder, filename))
    img.save(os.path.join(output_folders["ground_truth"], filename))
    
    src_path = Image.open(os.path.join(sar_folder, filename))
    img.save(os.path.join(output_folders["sar"], filename))

    print(f"Saved {filename} to output folders.")

Saved tile_2494.png to output folders.
Saved tile_2758.png to output folders.
Saved tile_2844.png to output folders.
Saved tile_3294.png to output folders.
Saved tile_1876.png to output folders.
Saved tile_3113.png to output folders.
Saved tile_2490.png to output folders.
Saved tile_2755.png to output folders.
Saved tile_2938.png to output folders.
Saved tile_3382.png to output folders.
Saved tile_3385.png to output folders.
Saved tile_2667.png to output folders.
Saved tile_2933.png to output folders.
Saved tile_2403.png to output folders.
Saved tile_2669.png to output folders.
Saved tile_1514.png to output folders.
Saved tile_2319.png to output folders.
Saved tile_3023.png to output folders.
Saved tile_3650.png to output folders.
Saved tile_2579.png to output folders.
Saved tile_2934.png to output folders.
Saved tile_3025.png to output folders.
Saved tile_3029.png to output folders.
Saved tile_2133.png to output folders.
Saved tile_3651.png to output folders.
Saved tile_1956.png to ou

In [ ]:
sar_folder = "../data_cd/sar"
optical_folder = "../data_cd/optical"
ground_truth_folder = "../data_cd/ground_truth"

filenames = sorted(os.listdir(sar_folder))
total = len(filenames)
index = 0

def show_next_image():
    global index, img_label1, img_label2, img_label3

    if index >= total:
        status_label.config(text="Done reviewing all images.")
        keep_button.config(state=tk.DISABLED)
        delete_button.config(state=tk.DISABLED)
        return

    filename = filenames[index]
    status_label.config(text=f"{index+1}/{total}: {filename}")

    paths = [
        os.path.join(optical_folder, filename),
        os.path.join(sar_folder, filename),
        os.path.join(ground_truth_folder, filename)
    ]

    images = [Image.open(p).convert("RGB").resize((256, 256)) for p in paths]
    tk_images = [ImageTk.PhotoImage(img) for img in images]

    for label, tk_img in zip([img_label1, img_label2, img_label3], tk_images):
        label.config(image=tk_img)
        label.image = tk_img

def keep_image():
    global index
    index += 1
    show_next_image()

def delete_image():
    global index
    filename = filenames[index]
    os.remove(os.path.join(optical_folder, filename))
    os.remove(os.path.join(sar_folder, filename))
    os.remove(os.path.join(ground_truth_folder, filename))
    index += 1
    show_next_image()

root = tk.Tk()
root.title("Image Reviewer")

status_label = tk.Label(root, text="Loading...")
status_label.pack()

frame = tk.Frame(root)
frame.pack()

img_label1 = tk.Label(frame)
img_label1.grid(row=0, column=0, padx=5)

img_label2 = tk.Label(frame)
img_label2.grid(row=0, column=1, padx=5)

img_label3 = tk.Label(frame)
img_label3.grid(row=0, column=2, padx=5)

btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

keep_button = tk.Button(btn_frame, text="Keep", width=20, command=keep_image)
keep_button.grid(row=0, column=0, padx=10)

delete_button = tk.Button(btn_frame, text="Delete", width=20, command=delete_image)
delete_button.grid(row=0, column=1, padx=10)

show_next_image()
root.mainloop()